##bTree 

### bNode

In [ ]:
class bNode:
    
    def __init__(self, key, lNode=None, rNode=None, parent=None, keySetSize=5):
        
        self.parent = parent
        self.keySetSize = keySetSize
        self.keySet = []
        self.childSet = []
        
        self.keySet.append(key)
        self.childSet.append(lNode)
        self.childSet.append(rNode)
    
        
    def bNode_getChildIndex(self, child):                
        return self.childSet.index(child)
    
    def bNode_isLeaf(self):
        for x in self.childSet:
            if x != None:
                return False
        return True
    
    # node에 키 삽입 
    def bNode_insertKey(self, key, lNode=None, rNode=None):
        
        if lNode == None and rNode == None:
            leaf = True
            if self.bNode_isLeaf() == False :            
                return False
        else:
            leaf = False
            
        
        index = 0
        for x in self.keySet:
            if key < x : 
                break
            else:
                index +=1
        
        self.keySet.insert(index, key)
        self.childSet.insert(index, lNode)
        if leaf == False:
            self.childSet[index+1]=rNode
        
        return True    
    
    # node에 저장된 key list에서 오른쪽 마지막 원 값을 빼낸다.
    def bNode_getRightMostKeyOfLeafNode(self):
        index = self.keySet.__len__() -1
               
        key = self.keySet[index]
          
        del self.keySet[index]        
        del self.childSet[index+1]
        
        return key
    
    # node에 저장된 key list에서 왼쪽 첫번째 원 값을 빼낸다.
    def bNode_getLeftMostKeyOfLeafNode(self):

        key = self.keySet[0]  
        
        del self.keySet[0]
        del self.childSet[0]
        
        return key
    
        
    def bNode_separate(self):
        
        if self.keySet.__len__() < 3 :
            # not overflow
            return False, 0, None, None
        
        index = int(self.keySet.__len__() /2)
        
        key = self.keySet[index]
        
        lNode=bNode(None)
        lNode.keySet = self.keySet[     :index]
        lNode.childSet = self.childSet[ :index+1]
        for node in lNode.childSet:
            if node != None: #self가 leaf node가 아니면
                node.parent = lNode
        
        rNode=bNode(None)
        rNode.keySet = self.keySet[index+1      :]
        rNode.childSet = self.childSet[index+1  :]   
        for node in rNode.childSet:
            if node != None:  node.parent = rNode         
        
        return True, key, lNode, rNode
    
    def bNode_deleteKeyfromLeaf(self, key):
        
        if False == self.bNode_isLeaf():
            return False
        
        try:
            index = self.keySet.index(key)
        except:
            return False
        
        del self.keySet[index]
        del self.childSet[index]
        
        return True

## bTree

In [ ]:
class bTree:
    
    def __init__(self, key, keySetSize=5):
        
        self.root = bNode(key, None, None, None, keySetSize)  
    
    def bTree_findNode(self, key):
        
        node = self.root
        
        while node != None :
            try:
                index = node.keySet.index(key)
                break;
            except:
                index=0  
                for x in node.keySet:
                    if key < x:
                        node = node.childSet[index]
                        break;
                    else:
                        index += 1
                if key > x:
                    node = node.childSet[index+1]
        return node
    
    
    #self 를 subroot로 간주하고, key를 삽입할 leaf를 탐색한다.
    def bTree_findLeaftoInsertKey(self, key):
        
        node = self.root       
        
        #leaf node 탐색 
        while node != None :
            # default 로 마지막 link를 저장한다.
            childNodeLink = node.childSet[node.childSet.__len__()-1]
            
            index=0
            for x in node.keySet :
                if key == x : # key를 삽입하지 않고 종료
                    return False, node
                elif key < x :
                    # node에서 key보다 큰 첫번째 index를 찾았다 
                    # 즉, keySet[index-1] < key < keySet[index]를 만족한다.
                    # 이때, 관련된 link는 childSet[index]가 된다.
                    childNodeLink = node.childSet[index]                    
                    break
                else:
                    index +=1
                
            if childNodeLink != None :
                # 앞서 찾은 child Link가 None이 아니면, 
                # 해당 child node를 대상으로 탐색을 계속한다.
                node = childNodeLink
            else:
                #현재 node가 key를 삽입할 최하위 노드이다.
                break
        
        return True, node
        
    def bTree_transmitKeytoSiblingLeaf(self, node):
        parent = node.parent
        
        index = parent.bNode_getChildIndex(node)
        size = parent.keySetSize
        
        sibling = None
        #left sibling
        if index > 0:  
            if parent.childSet[index-1].keySet.__len__() < size:
                sibling = parent.childSet[index-1]
                key = node.bNode_getLeftMostKeyOfLeafNode()
                sibling.keySet.append(parent.keySet[index-1])
                sibling.childSet.append(None)
                index -= 1   #to insert a key into parent
        
        # right sibling
        if sibling == None and index < parent.childSet.__len__()-1:
            if parent.childSet[index+1].keySet.__len__() < size: 
                sibling = parent.childSet[index+1]
                key = node.bNode_getRightMostKeyOfLeafNode()
                sibling.keySet.insert(0, parent.keySet[index])
                sibling.childSet.append(None)
        
        
        if sibling == None :
            return False
          
        parent.keySet[index] = key
        return True 
    
    def bTree_handleOverflow(self, node):
        
        # node overflow 확인 
        if node.keySet.__len__() <= node.keySetSize:
            return True
        
        parent = node.parent
        
        result = False    
        if node.bNode_isLeaf() == True and parent != None : 
            # transmit key to a sibling node 
            # only if (node is neither leaf node nor root)        
            result = self.bTree_transmitKeytoSiblingLeaf(node)
            
        if result == False:        
            # node will be separated
            _ , key, lnode, rnode = node.bNode_separate()
            
            if parent == None: # node is root
                parent = bNode(key)
                parent.childSet=[lnode,rnode]
                self.root = parent
                lnode.parent = parent
                rnode.parent = parent
                result = True
            
            else:            
                #level이 2이상이면 parent 관리에 문제가 발생한다.
                #bNode_separate()에서 lnonde와 rnode의 child의 parent를 다시 설정해야 한다.     
                lnode.parent = parent
                rnode.parent = parent
                parent.bNode_insertKey(key,lnode,rnode)
                
                if parent.keySet.__len__() > parent.keySetSize:
                    result = self.bTree_handleOverflow(parent)
        
        return result
    
    
    def bTree_insertKeyintoLeaf(self, key):
    
        result, leaf = self.bTree_findLeaftoInsertKey(key)  
        
        if result == True :            
        
            leaf.bNode_insertKey(key)
            
            self.bTree_handleOverflow(leaf)

        return result, leaf
    
    #underflow를 처리하기 위한 방안 1: 재분배 
    def __bTree_redistributeKeybetweenLeaves(self, node):
        
        # 이 함수는 node가 leaf 일 때만 사용할 수 있다.
        if node.bNode_isLeaf() == False:
            return False
        
        # 이 함수는 node가 root이면 사용하지 않는다.
        # 단, node가 root이면, underflow 처리를 하지 않도록 하기 이하여 True를 리턴한다.
        parent = node.parent
        if parent == None: # node is root
            return True
        
        index  = parent.bNode_getChildIndex(node)
        size   = int(parent.keySetSize/2)
        
        sibling = None 
        if index > 0:  
            if parent.childSet[index-1].keySet.__len__() > size:
                sibling = parent.childSet[index-1]
                key = sibling.bNode_getRightMostKeyOfLeafNode()
                node.keySet.insert(0,parent.keySet[index-1])
                node.childSet.append(None)
                index -= 1   # to insert key into parent
        
        # right sibling
        if sibling == None and index < parent.childSet.__len__()-1:
            if parent.childSet[index+1].keySet.__len__() > size: 
                sibling = parent.childSet[index+1]
                key = sibling.bNode_getLeftMostKeyOfLeafNode()                
                node.keySet.append(parent.keySet[index])                
                node.childSet.append(None)
        
        # it cannot get a key from its siblings
        if sibling == None :
            return False
          
        parent.keySet[index] = key
        
        return True 
    
    #underflow를 해결하기 위한 방안 2: 병
    def __bTree_mergeWithSiblingLeaf(self, node):
        
        parent = node.parent
        if parent == None: # node is root
            return True, None
        
        index   = parent.bNode_getChildIndex(node)
        size    = node.keySetSize
        nodeSize    = node.keySet.__len__()
        
        sibling = None   
        keyIndex        = index
        siblingIndex    = index+1     
        if index > 0 and parent.childSet[index-1].keySet.__len__() +nodeSize+1<= size :
            sibling = parent.childSet[index-1]
            keyIndex        = index-1
            siblingIndex    = index-1
        
        if index < parent.childSet.__len__()-1 and parent.childSet[index+1].keySet.__len__() +nodeSize+1<= size :
            # if case A or case B
            if sibling == None:
                sibling = parent.childSet[index+1]
            elif parent.childSet[index+1].keySet.__len__() < parent.childSet[index-1].keySet.__len__():
                sibling = parent.childSet[index+1]
                
        
        # 재분배 우선  --> 병합으로 인하여 overflow가 발생할 수 있기 때문에
        if sibling == None:
            return False, None
                
        key = parent.keySet[keyIndex]
        
        #병합되는 sibling 노드의 child들의 parent를 변경해야 한다.
        for x in sibling.childSet:
            if x != None: 
                x.parent = node
        
        #node에 key와 sibling을 병합한다.        
        if keyIndex < index: 
            #왼쪽 sibling이랑 병합: node = sibling + key + node
            node.keySet.insert(0,key)
            node.keySet = sibling.keySet + node.keySet
            node.childSet = sibling.childSet + node.childSet            
            
        else: 
            #오른쪽 sibling이랑 병합: node = node + key + sibling
            node.keySet.append(key)
            node.keySet     += sibling.keySet          
            node.childSet   += sibling.childSet    
            
        #parent의 수정
        del parent.keySet[keyIndex]
        del parent.childSet[siblingIndex]
        
        # tree의 root가 변경된 경우
        if parent == self.root and parent.keySet.__len__() == 0:
            self.root = node
            node.parent = None
            
                
        return True, parent    
    
    def bTree_handleUnderflow(self, node):
        
        if node.keySet.__len__() >= int(node.keySetSize / 2) :
            return True
        
        
        result = False

        #case 1: 재분배
        result = self.__bTree_redistributeKeybetweenLeaves(node)
            
        #case 2: 병합
        if result == False: 
            result, parent = self.__bTree_mergeWithSiblingLeaf(node)
            if result == True and parent != None:
                self.bTree_handleUnderflow(parent)
        
        return result
    
    def bTree_findSmallestKeyinRightSubset(self, key): 
        
        node = self.bTree_findNode(key)
        
        if node == None:
            return False, None          
            
        if node.bNode_isLeaf() == True:
            return True, node, key
        
        index = node.keySet.index(key)
        index += 1   #오른쪽 child의 index
        node = node.childSet[index]
        
        while node.childSet[0] != None:
            node = node.childSet[0]
        
        return True, node, node.keySet[0] 
    
    def bTree_deleteKey(self, key):
        
        node = self.bTree_findNode(key)
        
        if node == None:
            return True
        
        keyIndex = node.keySet.index(key)
        
        leaf = None
        
        if node.bNode_isLeaf() == True:
            leaf = node
            del leaf.keySet[keyIndex]
            del leaf.childSet[keyIndex]
            if leaf == self.root and leaf.keySet.__len__() == 0:
                pass
        else:
        
            result, leaf, smallestKey = self.bTree_findSmallestKeyinRightSubset(key)
            
            if result == False:
                return False
            
            node.keySet[keyIndex] = smallestKey
            del leaf.keySet[0]
            del leaf.childSet[0]
        
        
        if leaf != None:
            #underflow 처리
            self.bTree_handleUnderflow(leaf)
            
        
#            if __name__ == "__main__":
#                print(node, leaf)
        

## Test

In [ ]:
    test = 2
    
    #data=[5,10,1,13,6,16,9,21,31,25,30,20,55,44,33,22,11,60,61]
    data=[]
    for x in range(100):
        data.append(x)
    
    if test == 2:
        tree = bTree(data[0])
        for x in data[1:] :
            result, node = tree.bTree_insertKeyintoLeaf(x)
            print(x, ":", result, node)
        print(tree.root)
           
        result, node, key = tree.bTree_findSmallestKeyinRightSubset(47)
        
        print(node, key)
        
        #tree.bTree_deleteKey(47)
        
        #print(tree.root)
        
        for x in range(100):
            print("delete %d"%x)
            tree.bTree_deleteKey(x)
            
        print(tree.root)
        
    elif test == 1:
        node = bNode(5)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInLeaf(10)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInLeaf(1)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInLeaf(13)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInLeaf(7)
        print(node.keySet)
        print(node.childSet)
        print(node.keySet.__len__()) 
        node.childSet=[11,12,13,14,15,16]
        
        _, key, ln, rn = node.bNode_separate()   
        
        print(key)
        print(ln.keySet)
        print(ln.childSet)
        print(rn.keySet)
        print(rn.childSet)    
        
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInParent(0, 0, 0)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInParent(9, 9, 9)
        print(node.keySet)
        print(node.childSet)
        node.bNode_insertKeyInParent(17, 17, 17)
        print(node.keySet)
        print(node.childSet)
        